# Nettoyage de données

In [1]:
import pandas as pd
import numpy as np
import json
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk

In [2]:
!curl -XDELETE http://localhost:9200/movies/

{"acknowledged":true}
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    21  100    21    0     0     98      0 --:--:-- --:--:-- --:--:--    99


In [3]:
LOCAL = True
es_client = Elasticsearch(hosts=["localhost" if LOCAL else "elastic"])

with open('dataFin.json') as json_data:
    v = json.load(json_data)

#print(v)

def generate_data_exemple(documents):
    for docu in documents:
        yield {
            "_index": "movies",
            "_type": "movie",
            "_source": {k: v if v else None for k, v in docu.items()},
        }

bulk(es_client, generate_data_exemple(v))

(30, [])

In [4]:
!curl http://localhost:9200/_cat/indices?v

health status index  uuid                   pri rep docs.count docs.deleted store.size pri.store.size
yellow open   movies dIXwh49FROmRR6WNFsCo6A   1   1         30            0     14.2kb         14.2kb
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   204  100   204    0     0  10200      0 --:--:-- --:--:-- --:--:-- 10200


In [20]:
with open('dataFin.json') as json_data:
   v = json.load(json_data)

print(v)

[{'titre': 'La vie est belle', 'Evaluation spectateur': 8.2, 'image_urls': 'https://media.senscritique.com/media/000016162240/150/La_vie_est_belle.jpg'}, {'titre': 'Christmas Story', 'Evaluation spectateur': 6.2, 'image_urls': 'https://media.senscritique.com/media/000016127185/150/Christmas_Story.jpg'}, {'titre': 'Tout... sauf en famille', 'Evaluation spectateur': 4.6, 'image_urls': 'https://media.senscritique.com/media/000000080288/150/Tout_sauf_en_famille.jpg'}, {'titre': 'Super Noël', 'Evaluation spectateur': 5.7, 'image_urls': 'https://media.senscritique.com/media/000018248554/150/Super_Noel.jpg'}, {'titre': 'Hyper Noël', 'Evaluation spectateur': 4.9, 'image_urls': 'https://media.senscritique.com/media/000018252931/150/Hyper_Noel.jpg'}, {'titre': 'Super Noël méga givré (Super Noël 3)', 'Evaluation spectateur': 4.0, 'image_urls': 'https://media.senscritique.com/media/000005109764/150/Esprit_de_famille.jpg'}, {'titre': 'Le Grinch', 'Evaluation spectateur': 5.2, 'image_urls': 'https:/

In [17]:
QUERY = {
    "sort" : [
        { "Evaluation spectateur" : {"order" : "desc"}},
    ],
    "query" : {
        "match_all": {}
    }
}

result = es_client.search(index="movies", body=QUERY)
{elt['_source']['titre']:elt['_source']['image_urls']  for elt in result["hits"]["hits"]}

{'La vie est belle': 'https://media.senscritique.com/media/000016162240/150/La_vie_est_belle.jpg',
 'Le Noël de Mickey': 'https://media.senscritique.com/media/000019110917/150/A_la_maison_pour_Noel.jpg',
 "Maman, j'ai raté l'avion !": 'https://media.senscritique.com/media/000000025473/150/Maman_j_ai_encore_rate_l_avion.jpg',
 'Christmas Story': 'https://media.senscritique.com/media/000016127185/150/Christmas_Story.jpg',
 'Le Drôle de Noël de Scrooge': 'https://media.senscritique.com/media/000006328396/150/Richie_Rich_Meilleurs_Voeux.jpg',
 'Le Pôle Express': 'https://media.senscritique.com/media/000018279972/150/Marie_avant_Noel.jpg',
 'Super Noël': 'https://media.senscritique.com/media/000018248554/150/Super_Noel.jpg',
 "Maman, j'ai encore raté l'avion...": 'https://media.senscritique.com/media/000000010862/150/La_Fille_du_pere_Noel.jpg',
 'Le Sapin a les boules': 'https://media.senscritique.com/media/000006182927/150/Le_Drole_de_Noel_de_Scrooge.jpg',
 'This Christmas': 'https://media

In [11]:
def search_by_word(film):
    QUERY = {
        "query": {
            "multi_match" : {
                "query":    film,
                "fields": [ "titre", "overview" ] 
                }
            }
        }
    result = es_client.search(index="movies", body=QUERY)
    return [elt['_source']['titre']  for elt in result["hits"]["hits"]]

print(search_by_word('maman'))

["Maman, j'ai raté l'avion !", "Maman, j'ai encore raté l'avion..."]
